In [ ]:
import os

import earthaccess
import numpy as np
import pandas as pd
import xarray as xr
from netCDF4 import Dataset

earthaccess.login()

In [2]:
# Download some MODIS/Terra Thermal Anomalies/Fire 5-Min L2 Swath 1km data (100 files)
results = earthaccess.search_data(
    concept_id="C2271754179-LPCLOUD", count=100, temporal=("2015-06-01", "2015-12-31")
)
files = earthaccess.download(results, "earthaccess_data")

QUEUEING TASKS | : 100%|██████████| 100/100 [00:00<00:00, 20068.44it/s]
PROCESSING TASKS | : 100%|██████████| 100/100 [00:00<00:00, 362202.42it/s]
COLLECTING RESULTS | : 100%|██████████| 100/100 [00:00<00:00, 863025.51it/s]


In [3]:
# For each file, store the maximum fire mask value, whether all values could be loaded, and which variable loading failed on (if relevant)
results = pd.DataFrame()
cwd = os.getcwd()
for file in files:
    fh = Dataset(os.path.join(cwd, file), mode="r")
    data_values = np.unique(fh.variables["fire mask"][:].data)
    results.loc[file, "max fire mask value"] = np.max(data_values)
    try:
        for k in fh.variables.keys():
            _ = fh.variables[k][:]
        results.loc[file, "successful read"] = True
    except Exception:
        results.loc[file, "successful read"] = False
        results.loc[file, "failed on"] = k

In [4]:
results.tail(n=10)

,max fire mask value,successful read,failed on
earthaccess_data/MOD14.A2015152.0725.061.2021323235344.hdf,9.0,True,NaN
earthaccess_data/MOD14.A2015152.0730.061.2021323235229.hdf,9.0,True,NaN
earthaccess_data/MOD14.A2015152.0735.061.2021323232141.hdf,9.0,True,NaN
earthaccess_data/MOD14.A2015152.0740.061.2021323231851.hdf,8.0,True,NaN
earthaccess_data/MOD14.A2015152.0745.061.2021323232155.hdf,9.0,True,NaN
earthaccess_data/MOD14.A2015152.0750.061.2021323233357.hdf,9.0,True,NaN
earthaccess_data/MOD14.A2015152.0755.061.2021323235249.hdf,5.0,False,FP_line
earthaccess_data/MOD14.A2015152.0800.061.2021324000515.hdf,4.0,False,FP_line
earthaccess_data/MOD14.A2015152.0805.061.2021323233130.hdf,5.0,False,FP_line
earthaccess_data/MOD14.A2015152.0810.061.2021323233456.hdf,5.0,False,FP_line


MOD14 fire mask pixel classes
Value	Description
- 0 Not processed (missing input data)
- 1	Not processed (obsolete; not used since Collection 1)
- 2	Not processed (other reason)
- 3	Non-fire water pixel
- 4	Cloud (land or water)
- 5	Non-fire land pixel
- 6	Unknown (land or water)
- 7	Fire (low confidence, land or water)
- 8	Fire (nominal confidence, land or water)
- 9	Fire (high confidence, land or water)

In [5]:
# Find the maximum and minimum fire mask values grouped by whether all variables could be read
results.groupby(["successful read"]).agg({"max fire mask value": ["min", "max"]})

max fire mask value     
                                min  max
successful read                         
False                           4.0  5.0
True                            8.0  9.0

In [6]:
# Try reading the file into xarray
xr.open_dataset(files[0], engine="netcdf4")

RuntimeError: NetCDF: HDF error